In [1]:
from torch import nn, optim
import torch
from torchvision.models import vit_l_16, ViT_L_16_Weights
import os
import time
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode
from torchvision.datasets import ImageFolder

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print('Running on MPS')
elif torch.cuda.is_available():
    os.environ["USE_FLASH_ATTENTION"] = "1"
    device = torch.device("cuda:0")
    torch.backends.cuda.matmul.allow_tf32 = True
    
# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
    torch.backends.cudnn.allow_tf32 = True
    print('Running on CUDA')
else:
    device = torch.device("cpu")
    print('Running on CPU')

Running on CUDA


In [3]:


model = vit_l_16(weights = ViT_L_16_Weights.IMAGENET1K_V1, progress = True)
for param in model.parameters():
     param.requires_grad = False
model.heads = nn.Sequential(nn.Linear(1024, 4))

model.dropout= nn.Dropout(0.2)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [4]:

train_dataloader = torch.utils.data.DataLoader(
    ImageFolder("Brain_Tumor_Datasets_Classified\\train",transform=ViT_L_16_Weights.IMAGENET1K_V1.transforms()),
    batch_size=32, shuffle=True,pin_memory=True,num_workers=8)
test_dataloader = torch.utils.data.DataLoader(
    ImageFolder("Brain_Tumor_Datasets_Classified\\test",transform=ViT_L_16_Weights.IMAGENET1K_V1.transforms()),
    batch_size=32, shuffle=True,pin_memory=True,num_workers=8)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Brain_Tumor_Datasets_Classified\\train'

In [5]:
#### Train model
train_loss=[]
train_accuary=[]
test_loss=[]
test_accuary=[]

num_epochs = 120  
start_time = time.time()

for epoch in range(num_epochs): 
    print("Epoch {} running".format(epoch))
    model.train()    
    running_loss = 0.  
    running_corrects = 0 
 
    for i, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data).item()
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects / len(train_dataloader.dataset) * 100.
    train_loss.append(epoch_loss)
    train_accuary.append(epoch_acc)

    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time() -start_time))
    
    if epoch%5==0 or epoch==0 or epoch==num_epochs-1:
        model.eval()
        with torch.no_grad():
            running_loss = 0.
            running_corrects = 0
            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).item()
            epoch_loss = running_loss / len(test_dataloader.dataset)
            epoch_acc = running_corrects / len(test_dataloader.dataset) * 100.
            test_loss.append(epoch_loss)
            test_accuary.append(epoch_acc)
       
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time()- start_time))
        torch.save(model.state_dict(), f"vt{epoch}.pth")

Epoch 0 running


NameError: name 'train_dataloader' is not defined